# Interpolation of Cloud Atmosphere using scipy.interpolate.griddata

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
sns.set_style("whitegrid")

from scipy.interpolate import griddata

## Creating a interpolation graph for cloud in atmosphere

> Using a cloud in an atmosphere

In [ ]:
df_cloud = pd.read_csv("../create_atm/atmospheres/aerosol_cloud_atm_a70_0_0.csv", header = None, index_col = 0)

In [ ]:
df_cloud

In [ ]:
N_cloud = int(df_cloud.shape[1]/2)
effr_cloud = [4,8,10,16,20,32] # effr_cloud = [0.1,0.5,1,2,4,8,10,16,20,32] 
effr_aerosol = [0.02,0.04,0.06,0.08,0.1]

In [ ]:
for i in range(N_cloud):
    plt.plot(df_cloud[1+i], df_cloud[i+1+N_cloud])

plt.legend(effr_cloud, title='effective radius of cloud ($\mu$m)')
plt.xlabel('Reflection Function ($\lambda$ = 0.56 $\mu$m)')
plt.ylabel('Reflection Function ($\lambda$  = 1.59 $\mu$m)')
plt.show()

In [ ]:
fig, ax = plt.subplots()

for i in range(N_cloud):
    plt.plot(df_cloud[i+1], df_cloud[i+1+N_cloud])
# plt.plot(df_cloud_atm[5], df_cloud_atm[11])
# plt.plot(df_cloud_atm[6], df_cloud_atm[12])

plt.legend(effr_cloud, title='effective radius of cloud ($\mu$m)')
plt.xlabel('Reflection Function ($\lambda$ = 0.56 $\mu$m)')
plt.ylabel('Reflection Function ($\lambda$  = 1.59 $\mu$m)')
plt.title('$\mu_{0}$ = 1, $\mu$ = sqrt(3)/2')

> data manipulation to plot lines of constant optical depth

In [ ]:
dft_cloud = df_cloud.T

In [ ]:
dft_cloud

In [ ]:
dTau_range_cloud = []

for col in dft_cloud.columns:
    dTau_range_cloud.append(col)

print(dTau_range_cloud)
N_dTau_cloud = len(dTau_range_cloud)

In [ ]:
fig, ax = plt.subplots()

for i in range(N_dTau_cloud):
    x = dft_cloud[dTau_range_cloud[i]][0:N_cloud]
    y = dft_cloud[dTau_range_cloud[i]][N_cloud:2*N_cloud]
    
    plt.plot(x,y)

In [ ]:
fig, ax = plt.subplots()

for i in range(N_cloud):
    plt.plot(df_cloud[i+1], df_cloud[i+1+N_cloud])
# plt.plot(df_cloud_atm[5], df_cloud_atm[11])
# plt.plot(df_cloud_atm[6], df_cloud_atm[12])

plt.legend(effr_cloud, title='effective radius of cloud ($\mu$m)')
plt.xlabel('Reflection Function ($\lambda$ = 0.56 $\mu$m)')
plt.ylabel('Reflection Function ($\lambda$  = 1.59 $\mu$m)')

for i in range(N_dTau_cloud):
    x = dft_cloud[dTau_range_cloud[i]][:N_cloud]
    y = dft_cloud[dTau_range_cloud[i]][N_cloud:2*N_cloud]
    
    plt.plot(x,y, color = 'k')

## Bilinear Interpolation

In [ ]:
dft_numpy = dft_cloud.to_numpy()

x_effr = dft_numpy[0]
for i in range(N_cloud-1):
    x_effr = np.concatenate([x_effr, dft_numpy[i+1]])

y_effr = dft_numpy[N_cloud]
for i in range(N_cloud-1):
    y_effr = np.concatenate([y_effr, dft_numpy[i+1+N_cloud]])

z_effr = np.concatenate([np.ones(N_dTau_cloud)*4, np.ones(N_dTau_cloud)*8, np.ones(N_dTau_cloud)*10, np.ones(N_dTau_cloud)*16, np.ones(N_dTau_cloud)*20, np.ones(N_dTau_cloud)*32])
z_effr = np.log10(z_effr)

xi_effr = np.arange(0,0.3,0.0001)
yi_effr = np.arange(0,0.25,0.0001)
xi_effr,yi_effr = np.meshgrid(xi_effr,yi_effr)

zi_effr = griddata((x_effr,y_effr),z_effr,(xi_effr,yi_effr),method='linear')

# plot
plt.rc('font', family='serif')

fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(x_effr,y_effr,'k.')
# plt.xlabel('Reflection Function, $0.56 $\mu m$')
# plt.ylabel('Reflection Function ($\lambda$ = 1.59 $\mu$m)')
cntr1 = ax.contourf(xi_effr, yi_effr, 10**zi_effr, levels=14, cmap = 'plasma')

plt.colorbar(cntr1)



for i in range(N_cloud):
    plt.plot(df_cloud[i+1], df_cloud[i+1+N_cloud], color = 'k', linewidth = 1)


#plt.legend(effr_cloud, title='effective radius of cloud ($\mu$m)')
plt.xlabel('TOA Reflectance, $0.56 \mu m$')
plt.ylabel('TOA Reflectance, $1.59 \mu m$')

## optical depth
for i in range(N_dTau_cloud):
    x = dft_cloud[dTau_range_cloud[i]][:N_cloud]
    y = dft_cloud[dTau_range_cloud[i]][N_cloud:2*N_cloud]
    
    plt.plot(x,y, color = 'k',linestyle =':', linewidth = 2)



# plot details
# effective radii
plt.text(0.305, 0.221, '$r_{e}=4$')
plt.text(0.305, 0.172, '$8$')
plt.text(0.305, 0.16, '$10$')
plt.text(0.305, 0.145, '$16$')
plt.text(0.305, 0.132, '$20$')
plt.text(0.305, 0.108, '$32$')

# effective radii
plt.text(0.065, 0.049, r'$\tau = 4$')
plt.text(0.108, 0.072, '$8$')
plt.text(0.168, 0.093, '$16$')
plt.text(0.226, 0.1, '$32$')
plt.text(0.265, 0.1, '$64$')

# plot regions
plt.xlim(0.015,0.35)
plt.ylim(0.022,0.24)


fig.savefig('fig_interpolation.png', bbox_inches='tight')


plt.show()

### Testing the strength of the Effective Radius interpolation

In [ ]:
dft_numpy = dft_cloud.to_numpy()

x_effr = dft_numpy[0]
for i in range(N_cloud-1):
    if i != 1:
        x_effr = np.concatenate([x_effr, dft_numpy[i+1]])

y_effr = dft_numpy[N_cloud]
for i in range(N_cloud-1):
    if i != 1:
        y_effr = np.concatenate([y_effr, dft_numpy[i+1+N_cloud]])

z_effr = np.concatenate([np.ones(N_dTau_cloud)*4, np.ones(N_dTau_cloud)*8, np.ones(N_dTau_cloud)*16, np.ones(N_dTau_cloud)*20, np.ones(N_dTau_cloud)*32])
# np.ones(N_dTau_cloud)*10,
z_effr = np.log10(z_effr)

xi_effr = np.arange(0,0.3,0.0001)
yi_effr = np.arange(0,0.25,0.0001)
xi_effr,yi_effr = np.meshgrid(xi_effr,yi_effr)

zi_effr = griddata((x_effr,y_effr),z_effr,(xi_effr,yi_effr),method='linear')

# plot
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(x_effr,y_effr,'k.')
plt.xlabel('Reflection Function ($\lambda$ = 0.56 $\mu$m)')
plt.ylabel('Reflection Function ($\lambda$ = 1.59 $\mu$m)')
cntr1 = ax.contourf(xi_effr, yi_effr, 10**zi_effr, levels=14, cmap = 'plasma')

plt.colorbar(cntr1)



for i in range(N_cloud):
    if i!= 2:
        plt.plot(df_cloud[i+1], df_cloud[i+1+N_cloud], color = 'k', linewidth = 0.5)


#plt.legend(effr_cloud, title='effective radius of cloud ($\mu$m)')
plt.xlabel('TOA Reflectance ($\lambda$ = 0.56 $\mu$m)')
plt.ylabel('TOA Reflectance ($\lambda$  = 1.59 $\mu$m)')

## optical depth
for i in range(N_dTau_cloud):
    x = dft_cloud[dTau_range_cloud[i]][:N_cloud]
    y = dft_cloud[dTau_range_cloud[i]][N_cloud:2*N_cloud]
    
    plt.plot(x,y, color = 'k',linestyle =':', linewidth = 1.5)



# plot details
# effective radii
plt.text(0.305, 0.221, '$r_{e}=4$')
plt.text(0.305, 0.172, '$8$')
#plt.text(0.305, 0.16, '$10$')
plt.text(0.305, 0.145, '$16$')
plt.text(0.305, 0.132, '$20$')
plt.text(0.305, 0.108, '$32$')

# effective radii
plt.text(0.065, 0.049, r'$\tau = 4$')
plt.text(0.108, 0.072, '$8$')
plt.text(0.168, 0.093, '$16$')
plt.text(0.226, 0.1, '$32$')
plt.text(0.265, 0.1, '$64$')

# plot regions
plt.xlim(0.015,0.35)
plt.ylim(0.022,0.24)



plt.show()

In [ ]:
xi_test = dft_numpy[2]
yi_test = dft_numpy[2+N_cloud]
zi_test = griddata((x_effr,y_effr),z_effr,(xi_test,yi_test),method='linear')
print(10**(zi_test))
percentage_error = np.abs((10**(zi_test)-10)*10)
absolute_error = np.abs((10**(zi_test)-10))
retrieved_COD = 10**(zi_test)
print(percentage_error)
print(absolute_error)

In [ ]:
xi_test

In [ ]:
# interpolation strength figure

fig, ax = plt.subplots()
plt.plot(dTau_range_cloud, absolute_error)
plt.xlabel('COD')
plt.ylabel('Absolute Error on Retrieved CER')
plt.xscale('log', base=2)
plt.title('Absolute Error on Interpolation for $r_{e} = 10$')

#plot regions
#plt.xlim(0.5,64)
plt.xlim(0.125,128)


fig.savefig('fig_interpolation_strength.png', bbox_inches='tight')

fig.show()